# MapReduce Using `MRJob`: Part 2

## Job Posting Dataset

The sample dataset we will mainly use (`data/job-data/job-data-2018-09-*.txt`) for this tutorial contains job postings from one of the US job search websites. The data is stored with each row as a JSON document representing a job posting record. 

The example below shows a sample job postings from the data file. The sample record has been formatted with 4 spaces indentation. In the real file, each record is stored as a JSON document in one row.

*Example: JSON document of a job posting record*

```
{
    "industry": "Information Technology", 
    "datePosted": "2018-09-07", 
    "salaryCurrency": "USD", 
    "validThrough": "2018-10-07", 
    "empId": 671932, 
    "jobLocation": {
        "geo": {
            "latitude": "37.7623", 
            "@type": "GeoCoordinates", 
            "longitude": "-122.4145"
        }, 
        "@type": "Place", 
        "address": {
            "postalCode": "94110-2042", 
            "addressLocality": "San Francisco", 
            "@type": "PostalAddress", 
            "addressRegion": "CA", 
            "addressCountry": {
                "@type": 
                "Country", 
                "name": "US"
            }
        }
    }, 
    "estimatedSalary": {
        "@type": "MonetaryAmount", 
        "currency": "USD", 
        "value": {
            "maxValue": "202000", 
            "@type": "QuantitativeValue", 
            "unitText": "YEAR", 
            "minValue": "146000"
        }
    }, 
    "description": "<div><em>Generate insights and impact from data</em><em>.</em></div>\n<br/>\n<div>\n<div>We're looking for data scientists to join the Analytics team who are excited about applying their analytical skills to understand our users and influence decision making. If you are naturally data curious, excited about deriving insights from data, and motivated by having impact on the business, we want to hear from you.</div><br/>\n\n<div><strong>You will:</strong></div><div>\n\n\n<ul>\n<li>Work closely with product and business teams to identify important questions and answer them with data.</li>\n</ul>\n\n</div><br/>\n\n<div>\n\n\n<ul>\n<li>Apply statistical and econometric models on large datasets to: i) measure results and outcomes, ii) identify causal impact and attribution, iii) predict future performance of users or products.</li>\n</ul>\n\n</div><br/>\n\n<div>\n\n\n<ul>\n<li>Design, analyze, and interpret the results of experiments.</li>\n</ul>\n\n</div><br/>\n\n<div>\n\n\n<ul>\n<li>Drive the collection of new data and the refinement of existing data sources.</li>\n</ul>\n\n</div><br/>\n\n<div>\n\n\n<ul>\n<li>Create analyses that tell a \"story\" focused on insights, not just data.</li>\n</ul>\n\n</div><br/>\n\n<div><strong>We're looking for someone with:</strong></div><div>\n\n\n<ul>\n<li>3+ years experience working with and analyzing large data sets to solve problems.</li>\n</ul>\n\n</div><br/>\n\n<div>\n\n\n<ul>\n<li>A PhD or MS in a quantitative field (e.g., Economics, Statistics, Eng, Natural Sciences, CS).</li>\n</ul>\n\n</div><br/>\n\n<div>\n\n\n<ul>\n<li>Expert knowledge of a scientific computing language (such as R or Python) and SQL.</li>\n</ul>\n\n</div><br/>\n\n<div>\n\n\n<ul>\n<li>Strong knowledge of statistics and experimental design.</li>\n</ul>\n\n</div><br/>\n\n<div>\n\n\n<ul>\n<li>Ability to communicate results clearly and a focus on driving impact.</li>\n</ul>\n\n</div><br/>\n\n<div><strong>Nice to haves:</strong></div><div>\n\n\n<ul>\n<li>Prior experience with data-distributed tools (Scalding, Hadoop, Pig, etc).</li>\n</ul>\n\n</div><br/>\n\n<div><strong>You should include these in your application:</strong></div><div>\n\n\n<ul>\n<li>Resume and LinkedIn profile.</li>\n</ul>\n\n</div><br/>\n\n<div>\n\n\n<ul>\n<li>Description of the most interesting data analysis you've done, key findings, and its impact.</li>\n</ul>\n\n</div><br/>\n\n<div>\n\n\n<ul>\n<li>Link to or attachment of code you've written related to data analysis.</li>\n</ul>\n\n</div>\n</div>\n<br/>", 
    "hiringOrganization": {
        "@type": "Organization", 
        "sameAs": "www.stripe.com", 
        "name": "Stripe"
    },
    "@type": "JobPosting", 
    "jobId": 2280174543, 
    "@context": "http://schema.org", 
    "employmentType": "FULL_TIME", 
    "occupationalCategory": [
        "15-1111.00", 
        "Computer and Information Research Scientists"
    ], 
    "title": "Data Scientist"
}
```

Copy input data to HDFS:

In [1]:
!hdfs dfs -mkdir job-data/

mkdir: `job-data': File exists


In [2]:
!hdfs dfs -put ../data/job-data/* job-data/

put: `job-data/job-data-2018-09-08.txt': File exists
put: `job-data/job-data-2018-09-09.txt': File exists


## 3. Top N Pattern


Keys:

- Top n pattern aims to retrieve a relatively small number of records from a large data set according to a ranking scheme specified by user without sorting the entire data set.
- The subset needs to be small enough to fit into one single node and thus N should not be too large.

Applications:

- Anomaly detection
- Finding the top k records with the lowest/highest values

### 3.1 Top N Values

In our data set, each job contains two salaires (or no such fields if it's not available), `minValue` and `maxValue`. We want to find out the top N Salaries from all jobs. 

- To find the top n list across the entire dataset, we need to compare the values from all records, which means the key field becomes less useful and therefore we can assign `None` as the key, which will end up with one single reducer process the reduce job.

- To minimize the burden of the reducer and to maximize the parallelism, we can use a combiner to find a local top n list in each mapper container.

**Example**: Find the top N salaries from all jobs. 

To receive the top N salary list, we first compair `maxValue`, then `minValue`. 


- *Data flow*:

  - Input:`record`
  - $\quad\downarrow$
  - mapper:`<_, record> -> <_, (maxValue, minValue)>`
  - $\quad\downarrow$
  - combiner:`<_, local_top_n[(maxValue, minValue)]>`
  - $\quad\downarrow$
  - reducer:`<_, global_top_n[(maxValue, minValue)]>`
  - $\quad\downarrow$
  - Output:`top_n[(maxValue, minValue)]`
  
- *Features and highlights*:
  
  - Ignore key field in `mapper` so only one reducer will be involved  
  - `MRJob.reducer_init()` initializes top N sorted list
  - `MRJob.reducer()` inserts record into top N sorted list, alway truncate list to a length of N
  - `MRJob.reducer_final()` emits record from top N sorted list
  - Add `combiner_init`/`combiner`/`combiner_final` steps through `MrJob.steps()` to reduce data flow
  - To maintain a sorted list of size n, we use Python built-in heap sort algorithm to achieve O(log(n)) for each insertion operation.

In [3]:
%%file mr-jobs/3.1_top_n_value.py
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import JSONValueProtocol

import heapq


class MRTopNValue(MRJob):
    
    INPUT_PROTOCOL = JSONValueProtocol
    OUTPUT_PROTOCOL = JSONValueProtocol
        
    def configure_args(self):
        super().configure_args()
        self.add_passthru_arg('--top_n', type=int)
        
    def mapper(self, _, value):
        try:
            max_ = float(value['estimatedSalary']['value']['maxValue'])
            min_ = float(value['estimatedSalary']['value']['minValue'])
        except (KeyError, ValueError):
            pass
        else:
            yield _, (max_, min_)
    
    def reducer_init(self):
        if self.options.top_n < 1:
            raise ValueError('Invalid top_n value')
        self.top_n = []
        
    def reducer(self, _, values):
        for value in values:
            if len(self.top_n) < self.options.top_n:
                heapq.heappush(self.top_n, value)
            else:
                heapq.heappushpop(self.top_n, value)
                
    def reducer_final(self):
        for value in self.top_n:
            yield None, value
            
    def steps(self):
        return [MRStep(mapper=self.mapper,
                       combiner_init=self.reducer_init,
                       combiner=self.reducer,
                       combiner_final=self.reducer_final,
                       reducer_init=self.reducer_init,
                       reducer=self.reducer,
                       reducer_final=self.reducer_final)]


if __name__ == '__main__':
    MRTopNValue.run()

Writing mr-jobs/3.1_top_n_value.py


- Test locally:

In [4]:
!python3 mr-jobs/3.1_top_n_value.py ../data/job-data/* --output-dir mr-output --top_n 10

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/3.hadoop.20180926.223435.193694
job output is in mr-output
Removing temp directory /tmp/3.hadoop.20180926.223435.193694...


- Run on your Hadoop cluster:

In [5]:
!hdfs dfs -rm -r hdfs:///user/hadoop/mr-output

Deleted hdfs:///user/hadoop/mr-output


In [6]:
!python3 mr-jobs/3.1_top_n_value.py \
-r hadoop hdfs:///user/hadoop/job-data/ \
    --output-dir mr-output/ --top_n 10

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /usr/local/hadoop-2.8.4/bin...
Found hadoop binary: /usr/local/hadoop-2.8.4/bin/hadoop
Using Hadoop version 2.8.4
Looking for Hadoop streaming jar in /usr/local/hadoop-2.8.4...
Found Hadoop streaming jar: /usr/local/hadoop-2.8.4/share/hadoop/tools/lib/hadoop-streaming-2.8.4.jar
Creating temp directory /tmp/3.hadoop.20180926.223438.730336
Copying local files to hdfs:///user/hadoop/tmp/mrjob/3.hadoop.20180926.223438.730336/files/...
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar6624719287751284062/] [] /tmp/streamjob7229325374049235081.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Total input files to process : 2
  number of splits:2
  Submitting tokens for job: job_1537993323748_0019
  Submitted application application_1537993323748_0019
  The url to track the job: http://c8d937eb6693:80

### 3.2 Top N Records

With the top N Salaries being calculated in file `mr-output/part-00000`, now we want to find out which jobs offer those salaries. 

Since the top N values are considered small enough to fit into each node, it is preferable to distribute the file into those node where the mapper jobs run, rather than load data in the job configuration. This mechanism is called `Distributed Cache`. We can do it via `MRJob.add_file_arg()` method.

**Example**: Find all jobs that offer salaries from the top N salaries list. 

- *Data flow*:

  - mapper_init: fetch `top_n_list` to each mapper
  - $\quad\downarrow$
  - Input:`record`
  - $\quad\downarrow$
  - mapper:`<_, record>[if record contains salaries from top_n_list -> <_, record>]`
  - $\quad\downarrow$
  - Output:`record`
  
- *Features and highlights*:
  
  - `MRJob.add_file_arg('--cache')` sends an external file to Hadoop
  - To send a file to `cache` via command-line: `--cache <file_path>`
  - The cached file can then be accessd in script via: `MRJob.options.cache`
  - `MRJob.mapper_init()` fetches the top n list from the cached file to each mapper

In [7]:
%%file mr-jobs/3.2_top_n_job.py
from mrjob.job import MRJob
from mrjob.protocol import JSONValueProtocol

import os
import json

class MRTopNJob(MRJob):
    
    INPUT_PROTOCOL = JSONValueProtocol
    OUTPUT_PROTOCOL = JSONValueProtocol
        
    def configure_args(self):
        super().configure_args()
        self.add_file_arg('--cache')
    
    def mapper_init(self):
        self.cache = list()
        with open(self.options.cache, 'r') as f:
            for line in f:
                self.cache.append(tuple(json.loads(line)))
        
    def mapper(self, _, value):
        try:
            max_ = float(value['estimatedSalary']['value']['maxValue'])
            min_ = float(value['estimatedSalary']['value']['minValue'])
        except (KeyError, ValueError):
            pass
        else:
            if (max_, min_) in self.cache:
                yield _, value

if __name__ == '__main__':
    MRTopNJob.run()

Writing mr-jobs/3.2_top_n_job.py


- Test locally:

In [8]:
!python3 mr-jobs/3.2_top_n_job.py ../data/job-data/* --output-dir mr-output-jobs --cache mr-output/part-00000

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/3.hadoop.20180926.223523.328890
job output is in mr-output-jobs
Removing temp directory /tmp/3.hadoop.20180926.223523.328890...


- Run on your Hadoop cluster:

In [9]:
!hdfs dfs -rm -r hdfs:///user/hadoop/mr-output-jobs

Deleted hdfs:///user/hadoop/mr-output-jobs


In [10]:
!python3 mr-jobs/3.2_top_n_job.py \
-r hadoop hdfs:///user/hadoop/job-data/ \
    --output-dir mr-output-jobs/ \
    --cache hdfs:///user/hadoop/mr-output/part-00000

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /usr/local/hadoop-2.8.4/bin...
Found hadoop binary: /usr/local/hadoop-2.8.4/bin/hadoop
Using Hadoop version 2.8.4
Looking for Hadoop streaming jar in /usr/local/hadoop-2.8.4...
Found Hadoop streaming jar: /usr/local/hadoop-2.8.4/share/hadoop/tools/lib/hadoop-streaming-2.8.4.jar
Creating temp directory /tmp/3.hadoop.20180926.223527.144662
Copying local files to hdfs:///user/hadoop/tmp/mrjob/3.hadoop.20180926.223527.144662/files/...
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar1665997452787577890/] [] /tmp/streamjob432759349193367946.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Total input files to process : 2
  number of splits:2
  Submitting tokens for job: job_1537993323748_0020
  Submitted application application_1537993323748_0020
  The url to track the job: http://c8d937eb6693:808

## 4. Inverted Index

- Keys:
  - Indexing is a technique that is used frequently in almost all seach engine systems. Without an index, the search engine would scan every document in the corpus, which would require considerable time and computing power.
  - Inverted index is an index data structure storing a mapping from content, such as keywords, to its locations in a database file, or in a document or a set of documents. 
  - The purpose of an inverted index is to allow fast full text searches, at a cost of increased processing when a document is added to the database. 

**Example**: Generate an index from the data set to allow for faster searches on technique skills.

The skills we want to search for are listed below:

```
skillset = ['Java', 'JavaScript', 'C', 'C++', 'C#', 'Python', 'R', 'Bash',
            'MySQL', 'Postgresql', 'MongoDB', 'Html', 'Ruby', 'PHP', 
            'Swift', 'CSS', 'Julia', 'Golang', 'Github', 'Redis', 'Hadoop',
            'Spark', 'Hive', 'Pig', 'Spark', 'ElasticSearch', 'Kafka', 
            'Cassandra', 'AWS', 'GCP', 'Azure', 'Docker', 'kubernetes']
```

- *Data flow*:

  - mapper_init: define search regex pattern
  - Input:`record`
  - $\quad\downarrow$
  - mapper:`<_, record> -> find matching skills using regex -> <skill, jobId>`
  - $\quad\downarrow$
  - reducer:`<skill, [jobId]>`
  - $\quad\downarrow$
  - Output:`skill, [jobId]`
  
Note: Since no aggregation is performed throughout the entire MapReduce pipeline, a combiner in this example is redundant.

In [11]:
%%file mr-jobs/4_inverted_index.py
from mrjob.job import MRJob
from mrjob.protocol import JSONValueProtocol

import re


class MRIndexingSkills(MRJob):
    
    INPUT_PROTOCOL = JSONValueProtocol
    skillset = ['Java', 'JavaScript', 'C', 'C++', 'C#', 'Python', 'R', 'Bash',
                'MySQL', 'Postgresql', 'MongoDB', 'Html', 'Ruby', 'PHP', 
                'Swift', 'CSS', 'Julia', 'Golang', 'Github', 'Redis', 'Hadoop',
                'Spark', 'Hive', 'Pig', 'Spark', 'ElasticSearch', 'Kafka', 
                'Cassandra', 'AWS', 'GCP', 'Azure', 'Docker', 'kubernetes']
    
    def mapper_init(self):
        self.pattern = re.compile('|'.join(['(?<=\W){}(?=\W)'.format(re.escape(x)) for x in self.skillset]), 
                                  flags=re.IGNORECASE)
        
    def mapper(self, _, value):
        try:
            jobId, description = value['jobId'], value['description']
        except (KeyError, ValueError):
            pass
        else:
            skills = self.pattern.findall(description)
            for skill in skills:
                yield skill.capitalize(), jobId
    
    def reducer(self, key, values):
        yield key, list(values)
        

if __name__ == '__main__':
    MRIndexingSkills.run()

Writing mr-jobs/4_inverted_index.py


- Test locally:

In [12]:
!python3 mr-jobs/4_inverted_index.py ../data/job-data/* --output-dir mr-output

No configs found; falling back on auto-configuration
No configs specified for inline runner
Running step 1 of 1...
Creating temp directory /tmp/4_inverted_index.hadoop.20180926.223603.127426
job output is in mr-output
Removing temp directory /tmp/4_inverted_index.hadoop.20180926.223603.127426...


- Run on your Hadoop cluster:

In [13]:
!hdfs dfs -rm -r hdfs:///user/hadoop/mr-output

Deleted hdfs:///user/hadoop/mr-output


In [14]:
!python3 mr-jobs/4_inverted_index.py \
-r hadoop hdfs:///user/hadoop/job-data/ \
    --output-dir mr-output/

No configs found; falling back on auto-configuration
No configs specified for hadoop runner
Looking for hadoop binary in /usr/local/hadoop-2.8.4/bin...
Found hadoop binary: /usr/local/hadoop-2.8.4/bin/hadoop
Using Hadoop version 2.8.4
Looking for Hadoop streaming jar in /usr/local/hadoop-2.8.4...
Found Hadoop streaming jar: /usr/local/hadoop-2.8.4/share/hadoop/tools/lib/hadoop-streaming-2.8.4.jar
Creating temp directory /tmp/4_inverted_index.hadoop.20180926.223614.867498
Copying local files to hdfs:///user/hadoop/tmp/mrjob/4_inverted_index.hadoop.20180926.223614.867498/files/...
Running step 1 of 1...
  packageJobJar: [/tmp/hadoop-unjar8642215883446702203/] [] /tmp/streamjob8243278373205048200.jar tmpDir=null
  Connecting to ResourceManager at /0.0.0.0:8032
  Connecting to ResourceManager at /0.0.0.0:8032
  Total input files to process : 2
  number of splits:2
  Submitting tokens for job: job_1537993323748_0021
  Submitted application application_1537993323748_0021
  The url to track t